In [1]:
from google.cloud import bigquery
import os, csv, xmltodict, json

ModuleNotFoundError: No module named 'google'

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../creds.json"

In [3]:
PROJECT = 'eng-reactor-287421'
dataset = 'reference_data'
data_table = 'ice_nested'
map_table = 'ice_bq_map'

In [3]:
PROJECT = 'eng-reactor-287421'
dataset = 'zz_testing'
data_table = 'ice_nested'
map_table = 'ice_bq_map'

### ICE to BQ node mappings

In [5]:
file_dir = 'C:/Users/vcrom/Desktop/ficc/Files/'

In [9]:
with open(file_dir+'leaf_nodes.csv') as leaf_nodes:
    leaf_file = csv.reader(leaf_nodes)
    leaf_paths = list(leaf_file)
    leaf_nodes = leaf_paths[1:]
    

In [10]:
with open(file_dir+'repeated_record_nodes.csv') as repeated_record:
    repeated_record_file = csv.reader(repeated_record)
    repeated_record_paths = list(repeated_record_file)
    repeated_record_nodes = [entry[0] for entry in repeated_record_paths][1:]
    

In [11]:
address_key_list = [['/instrument/debt/put_details/put_type','text'],
                    ['/instrument/master_information/market_master/market/linked_markets/linked_market/effective_date','text'],
                    ['/instrument/debt/floating_rate_instruments/benchmark_code','text'],
                    ['/instrument/debt/additional_sink_details/sink_amount_type','text'],
                    ['/instrument/master_information/market_master/market/linked_markets/linked_market/effective_date','text']]

In [12]:
bad_address_list = ['/instrument/master_information/market_master/market']

### make schema for table storing mappings, push schema/values

In [13]:
bqclient = bigquery.Client(project=PROJECT,)
map_table_id = '{}.{}.{}'.format(PROJECT,dataset,map_table)

In [14]:
# delete table
#bqclient.delete_table(map_table_id, not_found_ok=True)

In [15]:
ice_bq_map_schema = [
    bigquery.SchemaField('ice_version','STRING'),
    bigquery.SchemaField('map_version','INTEGER'),
    bigquery.SchemaField('leaf_nodes','RECORD',mode='REPEATED', fields=[
        bigquery.SchemaField('path','STRING'),
        bigquery.SchemaField('type','STRING'),
        bigquery.SchemaField('mode','STRING'),
    ]),
    bigquery.SchemaField('repeated_record_nodes','STRING',mode='REPEATED'),
    bigquery.SchemaField('address_key','RECORD',mode='REPEATED', fields=[
        bigquery.SchemaField('path','STRING'),
        bigquery.SchemaField('key','STRING'),
    ]),
    bigquery.SchemaField('bad_address','STRING',mode='REPEATED'),
    ]

In [16]:
# create ICE to BQ mapping table
ice_bq_map_table = bigquery.Table(map_table_id, schema=ice_bq_map_schema)
ice_bq_map_table = bqclient.create_table(ice_bq_map_table)

In [17]:
# get ICE nested data table
ice_bq_map_table = bqclient.get_table(map_table_id)

In [18]:
ice_bq_map_table.schema

[SchemaField('ice_version', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('map_version', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('leaf_nodes', 'RECORD', 'REPEATED', None, (SchemaField('path', 'STRING', 'NULLABLE', None, (), None), SchemaField('type', 'STRING', 'NULLABLE', None, (), None), SchemaField('mode', 'STRING', 'NULLABLE', None, (), None)), None),
 SchemaField('repeated_record_nodes', 'STRING', 'REPEATED', None, (), None),
 SchemaField('address_key', 'RECORD', 'REPEATED', None, (SchemaField('path', 'STRING', 'NULLABLE', None, (), None), SchemaField('key', 'STRING', 'NULLABLE', None, (), None)), None),
 SchemaField('bad_address', 'STRING', 'REPEATED', None, (), None)]

In [19]:
ice_bq_map_update = [
    ('ICE47',0,leaf_nodes,repeated_record_nodes,address_key_list,bad_address_list)

]

In [20]:
bqclient.insert_rows(ice_bq_map_table,ice_bq_map_update)

[]

In [21]:
# get mapping schema from bigquery
map_query = """
SELECT * FROM {}.{}

           """.format(dataset,map_table)

In [22]:
bq_map_result = bqclient.query(map_query).result()

In [23]:
bq_map_result = list(bq_map_result)

In [24]:
ice_schema,map_version,leaf_nodes,repeated_record_nodes,address_key_list,bad_address_list = bq_map_result[0]

In [25]:
leaf_nodes = [[entry['path'],entry['type'],entry['mode']] for entry in leaf_nodes]

In [26]:
address_key_list = [[entry['path'],entry['key']] for entry in address_key_list]

### make schema from ICE dictionary csv

In [30]:
split_leaf_nodes = [[entry[0].split('/')[1:],[entry[1],entry[2]]] for entry in leaf_nodes]

In [31]:
repeated_leafs = [entry[0] for entry in leaf_nodes if entry[2]=='REPEATED']

In [32]:
# builds a nested dictionary from full ICE addresses of all leaf nodes
# leaf node contains [bq_types, repeated]

ice_schema_dict = {}
ice_schema_dict['ice_file_date']=['TIMESTAMP','NULLABLE']
for entry in split_leaf_nodes:
    entry_path, entry_details = entry
    current_dict_level = ice_schema_dict
    for current_index in range(len(entry_path)-1):
        if entry_path[current_index] not in current_dict_level:
            current_dict_level[entry_path[current_index]] = {}
        current_dict_level = current_dict_level[entry_path[current_index]]

    try:
        current_dict_level[entry_path[-1]]=entry_details
    except:
        print('/'.join(entry[0]))
        
        
        

In [35]:
# takes a nested dictionary, top level keys are RECORD nodes with value a dictionary with child nodes as keys
# terminal keys in nested dictionary contain a list [bq_type,repeated]
# path string builds up to check if a RECORD node needs to be mode='REPEATED'


def bq_schema_maker(tree_dict,repeaters,path_string=''):
    schema_list = []
    #add time stamp field
    #schema_list.append(bigquery.SchemaField('ice_file_date','TIMESTAMP',mode='NULLABLE'))
    # add from nested dictionary of fields
    for key in tree_dict.keys():
        if not isinstance(tree_dict[key],dict):
            bq_type, repeated = tree_dict[key]
            schema_list.append(bigquery.SchemaField(key,bq_type,mode=repeated))
        else:
            #how to get nullable vs repeated for top categories?
            new_path_string = path_string+'/'+key
            repeated_mode = "REPEATED" if new_path_string in repeaters else "NULLABLE"
            schema_list.append(bigquery.SchemaField(key,"RECORD",mode=repeated_mode,fields=bq_schema_maker(tree_dict[key],repeaters,new_path_string)))
    return schema_list

In [36]:
# add time stamp field to dict once
ice_schema_dict['ice_file_date']=('TIMESTAMP','NULLABLE')
bq_ice_schema = bq_schema_maker(ice_schema_dict,repeated_record_nodes)

### push nested ICE schema to BigQuery

In [37]:
bqclient = bigquery.Client(project=PROJECT,)
data_table_id = '{}.{}.{}'.format(PROJECT,dataset,data_table)

In [38]:
# delete table
#bqclient.delete_table(data_table_id, not_found_ok=True)

In [39]:
# create ICE nested data table
ice_nested_table = bigquery.Table(data_table_id, schema=bq_ice_schema)
ice_nested_table = bqclient.create_table(ice_nested_table)

In [ ]:
# update schema of existing ICE nested data table
ice_nested_table = bqclient.get_table(data_table_id)
ice_nested_table.schema = test_ice_schema
ice_nested_table = bqclient.update_table(ice_nested_table, ["schema"])


In [40]:
# get ICE nested data table
ice_nested_table = bqclient.get_table(data_table_id)

In [41]:
ice_nested_table.schema

[SchemaField('ice_file_date', 'TIMESTAMP', 'NULLABLE', None, (), None),
 SchemaField('instrument', 'RECORD', 'NULLABLE', None, (SchemaField('ice_file_date', 'TIMESTAMP', 'NULLABLE', None, (), None), SchemaField('id', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('debt', 'RECORD', 'NULLABLE', None, (SchemaField('ice_file_date', 'TIMESTAMP', 'NULLABLE', None, (), None), SchemaField('abs_details', 'RECORD', 'NULLABLE', None, (SchemaField('ice_file_date', 'TIMESTAMP', 'NULLABLE', None, (), None), SchemaField('abs_type', 'INTEGER', 'NULLABLE', None, (), None)), None), SchemaField('additional_sink_details', 'RECORD', 'NULLABLE', None, (SchemaField('ice_file_date', 'TIMESTAMP', 'NULLABLE', None, (), None), SchemaField('next_sink_date', 'DATE', 'NULLABLE', None, (), None), SchemaField('sink_amount_type', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('sink_frequency', 'INTEGER', 'NULLABLE', None, (), None)), None), SchemaField('additional_sink_schedule', 'RECORD', 'REPEATED', None, 